In [1]:
from Bagging import create_models, create_bags, evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
import random
from BaggingGA import BaggingGA
from BaggingSA import BaggingSA

In [2]:
seed = 42
np.random.seed(seed)
random.seed(seed)
data = datasets.load_digits()


X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [3]:


def evaluate_bagging_ga(reps, n_trees):
    res = []
    for r in range(reps):
        bagging_ga = BaggingGA(X=X_train, y=y_train,
                           n_trees=n_trees, max_iterations=150, mutation_rate=0.2, crossover_rate=0.7, population_size=100)
        models = bagging_ga.run(X_for_test=X_test, y_for_test=y_test)
        accuracy = evaluate(X_test, y_test, models=models)
        res.append(accuracy)
    return np.mean(res), np.std(res)


def evaluate_bagging(reps, n_trees):
    res = []
    for r in range(reps):
        bags = create_bags(X_train, n_trees)
        models = create_models(X=X_train, y=y_train, bags=bags)
        accuracy = evaluate(X_test, y_test, models=models)
        res.append(accuracy)
    return np.mean(res), np.std(res)

def evaluate_bagging_sa(reps, n_trees):
    res = []
    for r in range(reps):
        bagging_sa = BaggingSA(X=X_train, y=y_train,
                               T0=10, alpha=0.99, max_iterations=1000, n_trees=n_trees)
        models = bagging_sa.run(X_for_test=X_test, y_for_test=y_test)
        accuracy = evaluate(X_test, y_test, models=models)
        res.append(accuracy)
    return np.mean(res), np.std(res)

In [4]:
acc, std = evaluate_bagging_sa(1, 5)
print(f"Bagging SA: {acc} +/- {std}")

Iteration: 0, Temperature: 10.0000, Best: 0.6736, Fitness: 0.6736, New Fitness: 0.6736, Accuracy: 0.7417
Iteration: 1, Temperature: 9.9000, Best: 0.6736, Fitness: 0.6736, New Fitness: 0.6840, Accuracy: 0.7500
Iteration: 2, Temperature: 9.8010, Best: 0.6840, Fitness: 0.6840, New Fitness: 0.7014, Accuracy: 0.7361
Iteration: 3, Temperature: 9.7030, Best: 0.7014, Fitness: 0.7014, New Fitness: 0.6840, Accuracy: 0.7333
Iteration: 4, Temperature: 9.6060, Best: 0.7014, Fitness: 0.6840, New Fitness: 0.6944, Accuracy: 0.7667
Iteration: 5, Temperature: 9.5099, Best: 0.7014, Fitness: 0.6944, New Fitness: 0.6944, Accuracy: 0.7389
Iteration: 6, Temperature: 9.4148, Best: 0.7014, Fitness: 0.6944, New Fitness: 0.6875, Accuracy: 0.7500
Iteration: 7, Temperature: 9.3207, Best: 0.7014, Fitness: 0.6875, New Fitness: 0.7083, Accuracy: 0.7583
Iteration: 8, Temperature: 9.2274, Best: 0.7083, Fitness: 0.7083, New Fitness: 0.6736, Accuracy: 0.7361
Iteration: 9, Temperature: 9.1352, Best: 0.7083, Fitness: 0.673